In [1]:
!pip install dash

In [2]:
!pip install pandas
!pip install plotly
!pip install ta

In [3]:
!pip install pandas
!pip install finta
!pip install mplfinance
!pip install numpy
!pip install matplotlib
!pip install dash
!pip install plotly
!pip install scipy

In [3]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
from ta.trend import MACD

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from finta import TA
import mplfinance as mpf
import numpy as np
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import webbrowser
from scipy.signal import find_peaks

C:\Users\HP\AppData\Local\Temp\ipykernel_12120\1596782289.py:8: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\HP\AppData\Local\Temp\ipykernel_12120\1596782289.py:9: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [5]:
# Load dữ liệu giá và khối lượng
price = pd.read_excel("Price-Vol VN 2015-2023.xlsx", sheet_name='Price')
price = price[price['Code'] == "VT:AAA(P)"]
vol = pd.read_excel("Price-Vol VN 2015-2023.xlsx", sheet_name='Volume')
vol = vol[vol['Code'] == "VT:AAA(VO)"]

In [8]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import webbrowser


# Hàm chuyển định dạng dữ liệu
def stock_data(df):
    if df.empty:
        return df

    if len(df.columns) >= 3:
        df = pd.concat([df.columns.to_frame().T, df], ignore_index=True)
        df.columns = range(len(df.columns))
        df = df.drop(df.columns[[0, 1, 2]], axis=1)
        df = df.T
        df.columns = ["Ngày", "Giá"]
        df.reset_index(drop=True, inplace=True)
        df = df.set_index('Ngày')
    return df

# Chuẩn bị dữ liệu
price = stock_data(price)
vol = stock_data(vol)
vol.rename(columns={'Giá': 'Khối lượng'}, inplace=True)

price_vol = pd.concat([price, vol], axis=1)
price_vol = price_vol.fillna(0)

# Tính toán đường trung bình động
price_vol['MA_50'] = price_vol['Giá'].rolling(window=50).mean()
price_vol['MA_200'] = price_vol['Giá'].rolling(window=200).mean()

# Tính toán RSI (Relative Strength Index)
delta = price_vol['Giá'].diff(1)
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
price_vol['RSI'] = 100 - (100 / (1 + rs))

# Tính toán Stochastic Oscillator
stochastic_oscillator = ((price_vol['Giá'] - price_vol['Giá'].rolling(window=14).min()) /
                         (price_vol['Giá'].rolling(window=14).max() - price_vol['Giá'].rolling(window=14).min())) * 100
price_vol['Stochastic_Oscillator'] = stochastic_oscillator

# Tính toán Bollinger Bands
window = 20
price_vol['SMA'] = price_vol['Giá'].rolling(window=window).mean()
price_vol['Bollinger_Upper'] = price_vol['SMA'] + 2 * price_vol['Giá'].rolling(window=window).std()
price_vol['Bollinger_Lower'] = price_vol['SMA'] - 2 * price_vol['Giá'].rolling(window=window).std()

# Tính toán MACD (Moving Average Convergence Divergence)
macd = MACD(price_vol['Giá'], window_slow=26, window_fast=12, window_sign=9)
price_vol['MACD_Line'] = macd.macd()
price_vol['Signal_Line'] = macd.macd_signal()
price_vol['MACD_Histogram'] = price_vol['MACD_Line'] - price_vol['Signal_Line']
price_vol['MACD_Histogram'].fillna(method='ffill', inplace=True)

# Tạo ứng dụng Dash
app = dash.Dash(__name__)

# Tiêu đề và mô tả trang web
app.title = 'Chỉ số Chứng khoán và Biểu đồ Giá'
app.layout = html.Div([
    html.H1("Chỉ số Chứng khoán và Biểu đồ Giá", style={'text-align': 'center'}),
    html.P("Dữ liệu chứng khoán và các chỉ số kỹ thuật được biểu diễn bằng biểu đồ chuyên nghiệp.",
           style={'text-align': 'center'}),
    
    # Biểu đồ giá chính
    dcc.Graph(
        id='main-chart',
        figure={
            'data': [
                {'x': price_vol.index, 'y': price_vol['Giá'], 'type': 'line', 'name': 'Giá', 'line': {'color': 'blue'}},
                {'x': price_vol.index, 'y': price_vol['MA_50'], 'type': 'line', 'name': '50-day MA', 'line': {'color': 'orange', 'dash': 'dash'}},
                {'x': price_vol.index, 'y': price_vol['MA_200'], 'type': 'line', 'name': '200-day MA', 'line': {'color': 'green', 'dash': 'dash'}},
            ],
            'layout': {
                'title': 'Biểu Đồ Giá',
                'xaxis': {'rangeslider': {'visible': True}},
                'yaxis': {'title': 'Giá'},
            }
        }
    ),
    
    # Dropdown để chọn chỉ số
    html.Div([
        dcc.Dropdown(
            id='indicator-dropdown',
            options=[
                {'label': 'RSI', 'value': 'RSI'},
                {'label': 'Stochastic Oscillator', 'value': 'Stochastic_Oscillator'},
                {'label': 'SMA', 'value': 'SMA'},
                {'label': 'Bollinger Bands', 'value': 'Bollinger'},
                {'label': 'MACD', 'value': 'MACD'},
            ],
            value='RSI',
            style={'width': '50%', 'margin': 'auto'}
        ),
        
        # Biểu đồ chỉ số được cập nhật thông qua callback
        dcc.Graph(
            id='indicator-chart',
            style={'height': '400px'}
        )
    ])
])

# Callback để cập nhật biểu đồ chỉ số dựa trên lựa chọn người dùng
@app.callback(Output('indicator-chart', 'figure'), [Input('indicator-dropdown', 'value')])
def update_indicator_chart(selected_indicator):
    colors = {'RSI': 'purple', 'SMA': 'brown', 'Stochastic_Oscillator': 'red',
              'Bollinger': 'cyan', 'MACD_Line': 'magenta', 'Signal_Line': 'yellow', 'MACD_Histogram': 'grey'}

    if selected_indicator == 'RSI':
        return {
            'data': [
                {'x': price_vol.index, 'y': price_vol['RSI'], 'type': 'line', 'name': 'RSI', 'line': {'color': colors['RSI']}},
            ],
            'layout': {
                'title': 'Biểu Đồ RSI',
                'annotations': [
                    {
                        'text': 'RSI giúp đánh giá mức độ quá mua hoặc quá bán của một chứng khoán.',
                        'x': 0.5,
                        'y': 1.15,
                        'xref': 'paper',
                        'yref': 'paper',
                        'showarrow': False,
                        'font': {'size': 10},
                    },
                ],
                'yaxis': {'title': 'RSI'},
            }
        }
    elif selected_indicator == 'SMA':
        return {
            'data': [
                {'x': price_vol.index, 'y': price_vol['SMA'], 'type': 'line', 'name': 'SMA', 'line': {'color': colors['SMA']}},
            ],
            'layout': {
                'title': 'Biểu Đồ SMA',
                'annotations': [
                    {
                        'text': 'SMA (Đường Trung bình động) là một chỉ số kỹ thuật dùng để làm mịn giá chứng khoán theo thời gian.',
                        'x': 0.5,
                        'y': 1.15,
                        'xref': 'paper',
                        'yref': 'paper',
                        'showarrow': False,
                        'font': {'size': 10},
                    },
                ],
                'yaxis': {'title': 'SMA'},
            }
        }
    elif selected_indicator == 'Stochastic_Oscillator':
        return {
            'data': [
                {'x': price_vol.index, 'y': price_vol['Stochastic_Oscillator'], 'type': 'line', 'name': 'Stochastic Oscillator', 'line': {'color': colors['Stochastic_Oscillator']}},
            ],
            'layout': {
                'title': 'Biểu Đồ Stochastic Oscillator',
                'annotations': [
                    {
                        'text': 'Chỉ số Stochastic Oscillator đo lường mức độ xuyên suốt của xu hướng giá.',
                        'x': 0.5,
                        'y': 1.15,
                        'xref': 'paper',
                        'yref': 'paper',
                        'showarrow': False,
                        'font': {'size': 10},
                    },
                ],
                'yaxis': {'title': 'Stochastic Oscillator'},
            }
        }
    elif selected_indicator == 'Bollinger':
        return {
            'data': [
                {'x': price_vol.index, 'y': price_vol['Bollinger_Upper'], 'type': 'line', 'name': 'Upper Band', 'line': {'color': colors['Bollinger']}},
                {'x': price_vol.index, 'y': price_vol['SMA'], 'type': 'line', 'name': 'SMA', 'line': {'color': colors['SMA']}},
                {'x': price_vol.index, 'y': price_vol['Bollinger_Lower'], 'type': 'line', 'name': 'Lower Band', 'line': {'color': colors['Bollinger']}},
            ],
            'layout': {
                'title': 'Biểu Đồ Bollinger Bands',
                'annotations': [
                    {
                        'text': 'Bollinger Bands giúp đo lường biên độ của giá chứng khoán và đánh giá mức độ biến động.',
                        'x': 0.5,
                        'y': 1.15,
                        'xref': 'paper',
                        'yref': 'paper',
                        'showarrow': False,
                        'font': {'size': 10},
                    },
                ],
                'yaxis': {'title': 'Bollinger Bands'},
            }
        }
    elif selected_indicator == 'MACD':
        return {
            'data': [
                {'x': price_vol.index, 'y': price_vol['MACD_Line'], 'type': 'line', 'name': 'MACD Line', 'line': {'color': colors['MACD_Line']}},
                {'x': price_vol.index, 'y': price_vol['Signal_Line'], 'type': 'line', 'name': 'Signal Line', 'line': {'color': colors['Signal_Line']}},
                {'x': price_vol.index, 'y': price_vol['MACD_Histogram'], 'type': 'bar', 'name': 'MACD Histogram', 'marker': {'color': colors['MACD_Histogram']}},
            ],
            'layout': {
                'title': 'Biểu Đồ MACD',
                'annotations': [
                    {
                        'text': 'MACD (Moving Average Convergence Divergence) đo lường sự khác biệt giữa hai đường trung bình động.',
                        'x': 0.5,
                        'y': 1.15,
                        'xref': 'paper',
                        'yref': 'paper',
                        'showarrow': False,
                        'font': {'size': 10},
                    },
                ],
                'yaxis': {'title': 'MACD'},
                'xaxis': {'rangeslider': {'visible': True}},
            }
        }

if __name__ == '__main__':
    app.run_server(debug=True, port=8053)
    webbrowser.open_new("http://localhost:8053")


C:\Users\HP\AppData\Local\Temp\ipykernel_12120\4289044959.py:61: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

